https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/qdrant.html

In [3]:
import os
import qdrant_client
import sys
sys.path.insert(0, '../')
import local_secrets as secrets
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant, Pinecone
from langchain.document_loaders import TextLoader
from llama_index.readers.qdrant import QdrantReader
from llama_index import GPTListIndex
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
import ipywidgets as widgets
from IPython.display import clear_output
import pinecone
import langchain

In [4]:
os.environ['OPENAI_API_KEY'] = secrets.techstyle_openai_key
os.environ['PINECONE_API_KEY'] = secrets.techstyle_pinecone_api_key
print(langchain.__version__)

0.0.212


In [9]:
loader = TextLoader('../state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
qdrant = Qdrant.from_documents(docs, embeddings, url='http://localhost:6333', collection_name="state_of_the_union",)

In [94]:
reader = QdrantReader(host="localhost")
query_vector=[0.3, 0.7]*768
documents = reader.load_data(collection_name="github_llama_index", query_vector=query_vector, limit=5)
index = GPTListIndex.from_documents(documents)

In [88]:
[doc.extra_info for doc in documents]

[{'file_name': 'mock_embed_model.py',
  'file_path': 'llama_index/token_counter/mock_embed_model.py'},
 {'file_name': 'sql_query.py',
  'file_path': 'llama_index/indices/struct_store/sql_query.py'},
 {'file_name': 'base.py',
  'file_path': 'llama_index/indices/struct_store/base.py'},
 {'file_name': 'pandas_query.py',
  'file_path': 'llama_index/indices/struct_store/pandas_query.py'},
 {'file_name': 'schema.py',
  'file_path': 'llama_index/indices/query/schema.py'}]

In [92]:
len(documents[0].get_embedding())

1536

In [5]:
# https://qdrant.tech/articles/langchain-integration/
os.environ['OPENAI_API_KEY'] = secrets.techstyle_openai_key
llm = ChatOpenAI()
# pinecone
client = pinecone.init(api_key=secrets.techstyle_pinecone_api_key, environment='us-east-1-aws')
retriever = Pinecone(index=pinecone.Index('ssk'), embedding_function=OpenAIEmbeddings, text_key='text').as_retriever()
chain = load_qa_chain(llm=llm, chain_type='stuff') 

question = 'What are the classes in the llama_index package?'
question = 'How is the llama_index Node class used'
question = 'Show an example of how to use the llama_index Node class'
question = 'How can several llama_indexes be composed?'
question = 'Explain llama_index nodes'
# documents = retriever.get_relevant_documents(question)
#chain.run(input_documents=documents, question=question)

In [8]:
def on_button_clicked(b):
    with output:
        clear_output()
        documents = retriever.get_relevant_documents(question.value)
        print(chain.run(input_documents=documents, question=question.value))
output = widgets.Output()
question = widgets.Text()
button = widgets.Button(description='Chat')
button.on_click(on_button_clicked)
widgets.VBox([question, button, output])

In [25]:
print(chain.document_prompt)
print(len(documents))

PromptTemplate(input_variables=['page_content'], output_parser=None, partial_variables={}, template='{page_content}', template_format='f-string', validate_template=True)

In [25]:
index = pinecone.Index('ssk')
print(index.describe_index_stats())
vectorstore = Pinecone.from_existing_index(index_name='ssk', embedding=OpenAIEmbeddings(), namespace='github-llama-index')
documents = vectorstore.similarity_search('How can several llama_indexes be composed?')
len(documents)

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'github-llama-index': {'vector_count': 335}},
 'total_vector_count': 335}


4

In [34]:
llm = ChatOpenAI(model='gpt-3.5-turbo-16k')
retriever = Pinecone.from_existing_index(index_name='ssk', embedding=OpenAIEmbeddings(), namespace='github-llama-index').as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="map_reduce", retriever=retriever, return_source_documents=True)
result = qa({"query": 'How can several llama_indexes be composed? Show an example.'})
result['result']

'Here is an example of how several LlamaIndexes can be composed using the `CompositeIndex` class:\n\n```python\nfrom llama_index import LlamaIndex, CompositeIndex\n\n# Create individual indexes\nlist_index = LlamaIndex("list_index")\nvector_store_index = LlamaIndex("vector_store_index")\ntree_index = LlamaIndex("tree_index")\nkeyword_table_index = LlamaIndex("keyword_table_index")\n\n# Add indexes to the composite index\ncomposite_index = CompositeIndex()\ncomposite_index.add_index(list_index)\ncomposite_index.add_index(vector_store_index)\ncomposite_index.add_index(tree_index)\ncomposite_index.add_index(keyword_table_index)\n```\n\nIn this example, we create four individual LlamaIndexes: `list_index`, `vector_store_index`, `tree_index`, and `keyword_table_index`. These indexes can be of different types, such as the List Index, Vector Store Index, Tree Index, or Keyword Table Index.\n\nThen, we create a `CompositeIndex` and add the individual indexes to it using the `add_index` method.

In [ ]:
retriever = Qdrant(client=client, collection_name='github_llama_index', embeddings=OpenAIEmbeddings(), content_payload_key='text').as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="map_reduce", retriever=retriever, return_source_documents=True)
result = qa({"query": 'How can several llama_indexes be composed? Show an example.'})

0.0.212
